# Evaluating Responses from Image, Text, and Combined Indexes --> WIP

***This notebook works best with the `conda_python3` on the `ml.t3.large` instance***.

---

## Step 1. Setup

Install the required Python packages and import the relevant files.

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
# import necessary libraries to run this notebook
import os
import io
import sys
import json
import yaml
import glob
import boto3
import base64
import logging
import requests
import botocore
import sagemaker
import opensearchpy
import numpy as np
import pandas as pd
import globals as g
from PIL import Image
from pathlib import Path
from typing import List, Dict
from IPython.display import Image
from urllib.parse import urlparse
from botocore.auth import SigV4Auth
from pandas.core.series import Series
from sagemaker import get_execution_role
from botocore.awsrequest import AWSRequest
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from utils import get_cfn_outputs, get_text_embedding, get_llm_response, combined_llm_response

In [ ]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# global constants
CONFIG_FILE_PATH = "config.yaml"
# read the config yaml file
fpath = CONFIG_FILE_PATH
with open(fpath, 'r') as yaml_in:
    config = yaml.safe_load(yaml_in)
logger.info(f"config read from {fpath} -> {json.dumps(config, indent=2)}")

In [ ]:
!pygmentize globals.py

## Step 1. Fetch responses from the text, image and combined indexes 
---

Here, we fetch responses from the text index, image index to get a question bank created that acts as a curated dataset

In [ ]:

os.listdir(g.JSON_TEXT_DIR)

In [ ]:
text_index_list = os.listdir(g.JSON_TEXT_DIR)

text_index_list = [file for file in text_index_list if file.endswith('.json')]
text_index_list = [os.path.join(g.JSON_TEXT_DIR, file) for file in text_index_list]

print(text_index_list)

In [ ]:
# view the text only df
text_index_contents = []
for f in text_index_list:
    text_index_contents.append(json.loads(Path(f).read_text()))
text_index_df = pd.DataFrame(text_index_contents)
text_index_df.head(10)

In [ ]:
# view the image only df
image_index_contents = []
for f in img_index_files:
    image_index_contents.append(json.loads(Path(f).read_text()))
img_index_df = pd.DataFrame(image_index_contents)
img_index_df.head(10)

#### View all of the content from text only and image only indexes

In [ ]:
merged_df = pd.concat([text_index_df, img_index_df], ignore_index=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df.head(10)

## Clean Up
